In [2]:
import os
import numpy as np 
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from tqdm import tqdm
from shapely.geometry import Point,Polygon
import seaborn as sns
%matplotlib inline

data_root = './data-sample/taxi_sh'

读取数据

In [47]:
# 4316个出租车司机，一天在上海一天的行程
# len(os.listdir(data_root))
sh_od = pd.DataFrame()
for i,path in tqdm(enumerate(os.listdir(data_root))):
    per_czc = pd.read_csv(os.path.join(data_root,path))
    per_czc.columns = ['vehicle_num','date','longitude','latitude','speed','degree','status']
    sh_od = sh_od.append(per_czc,ignore_index=True)   

# sns.countplot('3',data=a_1) 
# a_1.columns = ['vehicle_num','date','longitude','latitude',]

4316it [23:25,  3.07it/s]


In [63]:
# 获取正常行驶的出租车编号,去除状态是0，1以外的情况
print (sh_od.shape)
sh_od = sh_od[sh_od['status'].between(0,1)]
print (sh_od.shape)
# 去除speed和degree这两列
sh_od = sh_od[['vehicle_num','date','longitude','latitude','status']]
print (sh_od.head())
sh_od.to_csv('./data-sample/sh_od.csv')

(4429131, 7)
(4429131, 7)
      vehicle_num                 date   longitude  latitude  status
2114        10004  2007-02-20 00:08:45  121.414583     31.16       1
2115        10004  2007-02-20 00:11:55  121.414583     31.16       1
2116        10004  2007-02-20 00:18:13  121.414583     31.16       1
2117        10004  2007-02-20 00:24:31  121.414583     31.16       1
2118        10004  2007-02-20 00:27:40  121.414583     31.16       1


# 初步进行数据分析
1. 判断出租车司机有多少个
2. 日期主要集中在什么时候
3. 经纬度的最大值与上海的行政区划比较

In [3]:
sh_od = pd.read_csv('./data-sample/sh_od.csv')
print (sh_od.head())
sh_od = sh_od[['vehicle_num','date','longitude','latitude','status']]
sh_od.info()

   Unnamed: 0  vehicle_num                 date   longitude  latitude  status
0        2114        10004  2007-02-20 00:08:45  121.414583     31.16       1
1        2115        10004  2007-02-20 00:11:55  121.414583     31.16       1
2        2116        10004  2007-02-20 00:18:13  121.414583     31.16       1
3        2117        10004  2007-02-20 00:24:31  121.414583     31.16       1
4        2118        10004  2007-02-20 00:27:40  121.414583     31.16       1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4429131 entries, 0 to 4429130
Data columns (total 5 columns):
vehicle_num    int64
date           object
longitude      float64
latitude       float64
status         int64
dtypes: float64(2), int64(2), object(1)
memory usage: 169.0+ MB


In [81]:
print ('最小日期为'+np.min(sh_od['date']))
print ('最大日期为'+np.max(sh_od['date']))

print (f'出租车数据最小经度为{np.min(sh_od.longitude)}')
print (f'出租车数据最大经度为{np.max(sh_od.longitude)}')
print (f'出租车数据最小维度为{np.min(sh_od.latitude)}')
print (f'出租车数据最大维度为{np.max(sh_od.latitude)}')

sh_xzgh = gpd.GeoDataFrame.from_file('./shapefile/shanghai_shp/区县界_area.shp')
bounds = sh_xzgh.geometry.apply(lambda x: x.bounds)
lat_min = bounds.apply(lambda x: x[1])
lat_max = bounds.apply(lambda x: x[3])
lon_min = bounds.apply(lambda x: x[0])
lon_max = bounds.apply(lambda x: x[2])
print (f'上海行政区划最小经度为{lon_min.min()}')
print (f'上海行政区划最大经度为{lon_max.max()}')
print (f'上海行政区划最小纬度为{lat_min.min()}')
print (f'上海行政区划最大纬度为{lat_max.max()}')

最小日期为2007-02-20 00:00:13
最大日期为2007-02-20 23:59:46
出租车数据最小经度为120.0345
出租车数据最大经度为121.991
出租车数据最小维度为30.0488
出租车数据最大维度为31.9646
上海行政区划最小经度为120.85172099982435
上海行政区划最大经度为121.97408500012276
上海行政区划最小纬度为30.682327999737417
上海行政区划最大纬度为31.866630000074537


In [118]:
# 经过筛选，一共有2822个出租车数据
sh_od.groupby('vehicle_num')['vehicle_num'].count().shape

(2822,)